In [2]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install Sastrawi

Note: you may need to restart the kernel to use updated packages.


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 13 10:45:23 2018

@author: MSI PANDU
DAFTAR API KEY TWITTER DULU
consumer_key='p89I417RqcklS0mkAdz8o9lXl'
consumer_secret='HApTkG7Ps8Y1lMkijYDBlbsHFh2JEJI4yBTB2uaHVnMwyOJC3a'
access_token='837224059-scTPV14pOLRcasGM7tvcD9ZEfZpYlSgCZS9ujvqH'
access_token_secret='JpzTZT6nwJWMIgUDCrFfTnpjdbyOTUdBnl5zo4GtAdxdZ'
"""
import tweepy
import re
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
consumer_key='YHrgCwdN231nhgb6uBMQtoujJ'
consumer_secret='mG2IfMA7CxUeJ4YYsnfcqhawSyXFGSJQcceKHjZT2xMZBtvV6T'
access_token='837224059-IisDL3Oi7byXcWbVgxMQDwL0X4I5ya7RMzrlhGJn'
access_token_secret='2Xawaks8HkJvtpWGqPA3dGlYK761uOclZrBerrqvGjwYW'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
searchQuery = '#covid'
retweet_filter='-filter:retweets'


data=[]
search_results = api.search(q=searchQuery+retweet_filter,lang='id', count=500)
for mention in search_results:
    tweet =mention.text
    tanggal= mention.created_at
    retweet=mention.retweet_count
    username=mention.author.screen_name
    hasil1=re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet)
    hasil2= hasil1.encode('ascii', 'ignore').decode('ascii')
    hasil3=' '.join(word for word in hasil2.split(' ') if not word.startswith('#'))
    hasil4=' '.join(word for word in hasil3.split(' ') if not word.startswith('@'))
    katadasar = stemmer.stem(str(hasil4))
    stop = stopword.remove(katadasar)
    hasil5=(" ".join(stop.split()))
    data.append([hasil5,tanggal,retweet,username])
datatw = pd.DataFrame(data)
datatw.index.name = 'Index'
datatw.to_csv("tweetlagii.csv",header=["Tweet","Tanggal","Jumlah Retweet","Username"])

In [13]:
df = pd.read_csv('tweet2.csv')
df

,Index,Tweet,Tanggal,Jumlah Retweet,Username
0,0,kawal yuk teman2 medical beau,2021-03-25 14:42:01,0,drjeffaloys
1,1,update kasus covid-19 indonesia kamis 25 maret...,2021-03-25 14:22:58,1,holopiscom
2,2,sobat polri ayo cegah tular covid-19 jaga jara...,2021-03-25 12:53:43,0,PatikrajaPolsek
3,3,negative report mandatory for entering bengalu...,2021-03-25 12:45:47,15,ndtv
4,4,berita 41 kluster institusi didik direkod seja...,2021-03-25 12:43:41,2,bernamadotcom
...,...,...,...,...,...
95,95,polri ayo cegah tular covid-19 jaga jarak a,2021-03-25 00:41:45,0,POLSEKBANJARNE1
96,96,top news koran rakyat merdeka bule langgar pro...,2021-03-25 00:20:00,2,RakyatMerdeka99
97,97,top news koran rakyat merdeka soal sekolah tat...,2021-03-25 00:10:00,2,RakyatMerdeka99
98,98,update data covid19 sulawesi barat tanggal 24 ...,2021-03-25 00:09:45,1,dinkes_sulbar


In [16]:
(df['Username'])

0         drjeffaloys
1          holopiscom
2     PatikrajaPolsek
3                ndtv
4       bernamadotcom
           ...       
95    POLSEKBANJARNE1
96    RakyatMerdeka99
97    RakyatMerdeka99
98      dinkes_sulbar
99     POLSEK_JTJOGJA
Name: Username, Length: 100, dtype: object

In [20]:
pip install scipy

In [22]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

tweets = pd.read_csv("tweet2.csv")
list(tweets.columns.values)

sentiment_counts = tweets.Sentimen.value_counts()
number_of_tweets = tweets.Index.count()
print(sentiment_counts)

from nltk.probability import FreqDist

fdist = FreqDist(tweets[(tweets.Sentimen == 'negatif')])
print(fdist.most_common(50))


count_vectorizer = CountVectorizer(ngram_range=(1,2))

vectorized_data = count_vectorizer.fit_transform(tweets.Tweet)
indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))



def sentiment2target(sentiment):
    return {
        'salah': 0,
        'benar': 1
    }[sentiment]

targets = tweets.Sentimen.apply(sentiment2target)

from sklearn.model_selection import train_test_split, KFold, cross_val_score

data_train, data_test, targets_train, targets_test = train_test_split(indexed_data, targets, test_size=0.4, random_state=0)
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier

kf =  KFold(n_splits=10, shuffle=True, random_state=1)

print("\n\nSVM:")

clf = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
cros= cross_val_score(clf, data_train, targets_train, scoring='accuracy', cv = kf)
print(cros)
print("Accuracy: " , cros.mean() * 100)

AttributeError: 'DataFrame' object has no attribute 'Sentimen'